# Lending Club Data Preparation

## This notebook is for Data Cleaning and Feature Engineering

**==============================================================================================================**

Before using this worksheet, the data needs to be processed. Let’s begin by processing the following categorical variables:

 

- application_type

- home_ownerhsip

- term

- verification_status

- loan_status

Be sure to create the same variables on both the Calibration Data sheet and the Validation Data sheet.

## Data Dictionary

| Field          | Description                                                                           |
|----------------|---------------------------------------------------------------------------------------|
|annual_inc |The self-reported annual income provided by the borrower during registration, in $1000s	|
|application_type |Indicates whether the loan is an individual application or a joint application with two co-borrowers	|
|collections_12_mths_ex_med |Number of collections in 12 months excluding medical collections	|
|delinq_2yrs |The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years	|
|dti |A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.	|
|home_ownership |The home ownership status provided by the borrower during registration. Our values are: RENT, OWN, MORTGAGE, OTHER.	|
|inq_last_6mths |The number of inquiries in past 6 months (excluding auto and mortgage inquiries)	|
|loan_status |Current status of the loan	|
|open_acc |The number of open credit lines in the borrower's credit file.	|
|pub_rec |Number of derogatory public records	|
|term |	The number of payments on the loan. Values are in months and can be either 36 or 60. |
|verification_status |Indicates if income was verified by LC, not verified, or if the income source was verified	|

**==============================================================================================================**

## Import Libraries

In [1]:
import numpy as np
from numpy import count_nonzero, median, mean
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
#import squarify

import datetime
from datetime import datetime, timedelta, date, time


#import os
#import zipfile
import scipy
from scipy import stats
from scipy.stats.mstats import normaltest # D'Agostino K^2 Test
from scipy.stats import boxcox
from collections import Counter

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, Binarizer 
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, RobustScaler

%matplotlib inline
#sets the default autosave frequency in seconds
%autosave 60 
sns.set_style('dark')
sns.set(font_scale=1.2)

plt.rc('axes', titlesize=9)
plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings('ignore')

# Use Feature-Engine library
import feature_engine

from feature_engine.imputation import AddMissingIndicator, CategoricalImputer, DropMissingData, MeanMedianImputer
from feature_engine.imputation import ArbitraryNumberImputer, RandomSampleImputer

from feature_engine.outliers import Winsorizer, ArbitraryOutlierCapper, OutlierTrimmer

from feature_engine.encoding import CountFrequencyEncoder, DecisionTreeEncoder, MeanEncoder, OneHotEncoder
from feature_engine.encoding import OrdinalEncoder, WoEEncoder, RareLabelEncoder, StringSimilarityEncoder

from feature_engine.discretisation import EqualWidthDiscretiser, EqualFrequencyDiscretiser, ArbitraryDiscretiser
from feature_engine.discretisation import DecisionTreeDiscretiser, EqualWidthDiscretiser

from feature_engine.datetime import DatetimeFeatures

from feature_engine.creation import CyclicalFeatures, MathFeatures, RelativeFeatures


pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format','{:.2f}'.format)

random.seed(0)
np.random.seed(0)
np.set_printoptions(suppress=True)

Autosaving every 60 seconds


## Data Quick Glance

In [2]:
df = pd.read_csv("validdata.csv")

In [19]:
df_ref = pd.read_csv("calibdata1.csv")

In [3]:
df.head()

,annualinc,applytype,collections,delinq,inq,openacc,dti,pubrec,homeowner,term,vstatus,lstatus
0,55.00,INDIVIDUAL,0,0,0,33,25.66,0,MORTGAGE,36 months,Not Verified,On Time
1,130.00,INDIVIDUAL,0,1,0,6,4.90,0,RENT,36 months,Verified,On Time
2,52.60,INDIVIDUAL,0,0,0,9,15.26,0,OWN,36 months,Verified,On Time
3,44.00,INDIVIDUAL,0,2,1,23,24.66,0,RENT,36 months,Source Verified,On Time
4,85.00,INDIVIDUAL,0,0,1,13,7.84,0,MORTGAGE,60 months,Verified,On Time


In [ ]:
df.info()

In [ ]:
df.dtypes.value_counts()

In [ ]:
# Descriptive Statistical Analysis
df.describe(include="all")

In [ ]:
# Descriptive Statistical Analysis
df.describe(include=["int", "float"])

In [ ]:
# Descriptive Statistical Analysis
df.describe(include="object")

In [ ]:
df.columns

In [ ]:
# Check target variable

df.lstatus.value_counts()

## Overall Visualization

In [ ]:
df.hist(bins=50, figsize=(20,50), layout=(len(df.columns),2), grid=False)
plt.suptitle('Histogram Feature Distribution', x=0.5, y=1.02, ha='center', fontsize=20)

plt.tight_layout()
plt.show()

In [ ]:
df.boxplot(figsize=(20,10), color='blue', fontsize=15)
plt.suptitle('BoxPlots Feature Distribution', x=0.5, y=1.02, ha='center', fontsize=20)

plt.tight_layout()
plt.show()

**==============================================================================================================**

**==============================================================================================================**

# Data Preprocessing

# Feature Engineering

  * **Feature selection**
    * Removing uninformative features
  * **Feature extraction**
    * Creating new features from existing features
  * **Feature transformation**
    * Modifying existing features to better suit our objectives
    * Encoding of categorical features as dummies
 
When modeling, best practice is to perform a rigorous examination of your data before beginning feature engineering and feature selection. This process is important. Not only does it help you understand your data, what it's telling you, and what it's _not_ telling you, but it also can give you clues that help you create new features. 

## Rename columns

In [ ]:
df.columns

In [ ]:
# remove special character
df.columns = df.columns.str.replace('_', '')

In [ ]:
df.columns

In [ ]:
df2.columns

In [ ]:
#Method 3: Using a new list of column names

# Creating a list of new columns
df_cols = ['annualinc', 'applytype', 'collections', 
           'delinq', 'inq', 'openacc', 'dti', 'pubrec', 
           'homeowner', 'term', 'term2', 'vstatus', 'lstatus'
          ]

# printing the columns
# before renaming
print(df.columns)

# Renaming the columns
df.columns = df_cols

# printing the columns
# after renaming
print(df.columns)


In [ ]:
df.head()

In [ ]:
#df.to_csv("validdata.csv", index=False)

## Rearrange columns

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
cols = ['annualinc', 'collections', 'delinq', 'inq', 'openacc', 'dti', 'pubrec', 
        'individual', 'mortgage', 'rent', 'own', 'other', 'term60mths', 'vstatusverified', 'vstatusnotverified',
        'lstatus'
        ]

In [ ]:
df = df[cols]

In [ ]:
df.head()

In [ ]:
df.lstatus.value_counts()

In [ ]:
#df.to_csv("calibdata1.csv", index=False)

**==============================================================================================================**

## Data Types

<p>The last step in data cleaning is checking and making sure that all data is in the correct format (int, float, text or other).</p>

In Pandas, we use:

<p><b>.dtype()</b> to check the data type</p>
<p><b>.astype()</b> to change the data type</p>


In [ ]:
df.dtypes

**==============================================================================================================**

## Treat Missing Values

<b>How to deal with missing data?</b>

<ol>
    <li>Drop data<br>
        a. Drop the whole row<br>
        b. Drop the whole column
    </li>
    <li>Replace data<br>
        a. Replace it by mean<br>
        b. Replace it by frequency<br>
        c. Replace it based on other functions
    </li>
</ol>

For easier detection of missing values, pandas provides the `isna()`, `isnull()`, and `notna()` functions. For more information on pandas missing values please check out this documentation).

There are several options for dealing with missing values. We will use 'Lot Frontage' feature to analyze for missing values.

1. We can drop the missing values, using `dropna()` method.

2. We can drop the whole attribute (column), that contains missing values, using the `drop()` method.

3. We can replace the missing values (zero, the mean, the median, etc.), using `fillna()` method.

In [4]:
df.isnull().sum()

annualinc      0
applytype      0
collections    0
delinq         0
inq            0
openacc        0
dti            0
pubrec         0
homeowner      0
term           0
vstatus        0
lstatus        0
dtype: int64

**==============================================================================================================**

## Treat Duplicate Values

In [5]:
df.duplicated(keep='first').sum()

0

In [ ]:
#identify duplicate rows
duplicateRows = df[df.duplicated(keep='last')]

In [ ]:
duplicateRows

In [ ]:
df.drop_duplicates(ignore_index=True, inplace=True)

In [ ]:
df

In [ ]:
#df.to_csv("calibdata.csv", index=False)

**==============================================================================================================**

In [ ]:
df.drop(['term2'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
#df.to_csv("validdata.csv", index=False)

**==============================================================================================================**

## Treat Outliers

In statistics, an outlier is an observation point that is distant from other observations. An outlier can be due to some mistakes in data collection or recording, or due to natural high variability of data points. How to treat an outlier highly depends on our data or the type of analysis to be performed. Outliers can markedly affect our models and can be a valuable source of information, providing us insights about specific behaviours.

There are many ways to discover outliers in our data. We can do Uni-variate analysis (using one variable analysis) or Multi-variate analysis (using two or more variables). One of the simplest ways to detect an outlier is to inspect the data visually, by making box plots or scatter plots. 

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
# Draw Box Plots

plt.figure(figsize=(20,7))
sns.boxplot(data=df.select_dtypes(include=np.number))
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x="annualinc", data=df, orient="h")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x="countfloorspreeq", orient="h", data=df)
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(data=df.age, orient="h")
plt.xlim(0, 100)
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(data=df.area, orient="h")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(data=df.videolikecount, orient="h")
plt.show()

In [ ]:
df.columns

### Removing outliers - outlier trimming

In [ ]:
outliertrimmer = OutlierTrimmer(capping_method="iqr", fold=1.5, tail="right", 
                                variables=['totalsteps' , 'calories'])

In [ ]:
outliertrimmer = OutlierTrimmer(capping_method="iqr", fold=1.5, tail="both", 
                                variables=['tripdistance', 'fareamount', 'tipamount', 'totalamount'])

In [ ]:
## Normal distribution
outliertrimmer = OutlierTrimmer(
    variables=["MedInc", "HouseAge", "Population"],
    capping_method="gaussian",
    tail="both",
    fold=3
)

In [ ]:
outliertrimmer = OutlierTrimmer(capping_method="quantiles", fold=0.05, tail="both", 
                                variables=['rainfall', 'windgustspeed', 'windspeed9am', 'windspeed3pm'])

In [ ]:
outliertrimmer.fit(df)

In [ ]:
outliertrimmer.left_tail_caps_

In [ ]:
outliertrimmer.right_tail_caps_

In [ ]:
df2 = outliertrimmer.transform(df)
df2

In [ ]:
df2.reset_index(drop=True, inplace=True)

In [ ]:
df2

In [ ]:
df.kmperdrivingday.hist();

In [ ]:
df2.kmperdrivingday.hist();

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(data=df2.starting, orient="h")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(data=df2.hours, orient="h")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(data=df2.hours, orient="h")
plt.show()

In [ ]:
df2.describe()

In [ ]:
#df2.to_csv("wellnessmod.csv", index=False)

## Capping / Censoring outliers

The Winsorizer() caps maximum and / or minimum values of a variable.

The Winsorizer() works only with numerical variables

In [ ]:
df.columns

In [ ]:
capper = Winsorizer(
    variables=["worst smoothness", "worst texture"],
    capping_method="gaussian",
    tail="both",
    fold=2,
)

In [ ]:
capper = Winsorizer(
    variables=['veryactiveminutes', 'fairlyactiveminutes', 'lightlyactiveminutes', 'sedentaryminutes'],
    capping_method="iqr",
    tail="right",
    fold=1.5,
)


In [ ]:
capper = Winsorizer(
    variables=["worst smoothness", "worst texture"],
    capping_method="quantiles",
    tail="both",
    fold=0.05,
)

In [ ]:
capper = Winsorizer(
    variables=["worst smoothness", "worst texture"],
    capping_method="mad",
    tail="both",
    fold=0.05,
)

In [ ]:
capper.fit(df)

In [ ]:
capper.left_tail_caps_

In [ ]:
capper.right_tail_caps_

In [ ]:
df2 = capper.transform(df)

In [ ]:
df2.describe()

In [ ]:
#df2.to_csv("wellnessmod.csv", index=False)

## ArbitraryOutlierCapper

The ArbitraryOutlierCapper caps the minimum and maximum values by a value determined by the user.

In [ ]:
df.columns

In [ ]:
# let's find out the maximum values
df[['countfloorspreeq', 'age', 'area', 'height']].max()

In [ ]:
df[['countfloorspreeq', 'age', 'area', 'height']].min()

In [ ]:
capper = ArbitraryOutlierCapper(
    max_capping_dict={'countfloorspreeq': 4.0, 'age': 70.0, 'area': 200.0, 'height': 10.0},
    min_capping_dict=None,
)


In [ ]:
capper = ArbitraryOutlierCapper(
    max_capping_dict=None,
    min_capping_dict={'fareamount': 0.00, 'totalamount': 0.00},
)


In [ ]:
capper.fit(df)

In [ ]:
capper.left_tail_caps_

In [ ]:
capper.right_tail_caps_

In [ ]:
df2 = capper.transform(df)

In [ ]:
df2

In [ ]:
df2.describe()

In [ ]:
#df2.to_csv("earthquakemod.csv", index=False)

**==============================================================================================================**

## One-hot encoding

There are three unique values: France, Spain, and Germany. Let's encode this data so it can be represented using Boolean features. We'll use a pandas function called `pd.get_dummies()` to do this.

When we call `pd.get_dummies()` on this feature, it will replace the `Geography` column with three new Boolean columns--one for each possible category contained in the column being dummied. 

When we specify `drop_first=True` in the function call, it means that instead of replacing `Geography` with three new columns, it will instead replace it with two columns. We can do this because no information is lost from this, but the dataset is shorter and simpler.  

In this case, we end up with two new columns called `Geography_Germany` and `Geography_Spain`. We don't need a `Geography_France` column. Why not? Because if a customer's values in `Geography_Germany` and `Geography_Spain` are both 0, we'll know they're from France! 

## One Hot Encoding - Feature-engine

Just like imputation, all methods of categorical encoding should be performed over the training set, and then propagated to the test set. 

Why? 

Because these methods will "learn" patterns from the train data, and therefore you want to avoid leaking information and overfitting. But more importantly, because we don't know whether in future / live data, we will have all the categories present in the train data, or if there will be more or less categories. Therefore, we want to anticipate this uncertainty by setting the right processes right from the start. We want to create transformers that learn the categories from the train set, and used those learned categories to create the dummy variables in both train and test sets.

In [20]:
df.columns

Index(['annualinc', 'applytype', 'collections', 'delinq', 'inq', 'openacc', 'dti', 'pubrec', 'homeowner', 'term', 'vstatus', 'lstatus'], dtype='object')

In [21]:
df.describe(include="object")

,applytype,homeowner,term,vstatus,lstatus
count,10000,10000,10000,10000,10000
unique,2,4,2,3,2
top,INDIVIDUAL,MORTGAGE,36 months,Source Verified,On Time
freq,9996,5024,6789,3780,9337


In [22]:
list(df.describe(include="object"))

['applytype', 'homeowner', 'term', 'vstatus', 'lstatus']

In [24]:
# set up encoder

encoder = OneHotEncoder(
    variables=['applytype', 'homeowner', 'term', 'vstatus'],  # alternatively pass a list of variables
    drop_last=True,  # to return k-1, use drop=false to return k dummies
)

In [25]:
# fit the encoder (finds categories)

encoder.fit(df)

OneHotEncoder(drop_last=True,
              variables=['applytype', 'homeowner', 'term', 'vstatus'])

In [26]:
# automatically found numerical variables

encoder.variables_

['applytype', 'homeowner', 'term', 'vstatus']

In [27]:
# we observe the learned categories

encoder.encoder_dict_

{'applytype': ['INDIVIDUAL'],
 'homeowner': ['MORTGAGE', 'RENT', 'OWN'],
 'term': ['36 months'],
 'vstatus': ['Not Verified', 'Verified']}

In [28]:
df_ref.columns

Index(['annualinc', 'collections', 'delinq', 'inq', 'openacc', 'dti', 'pubrec', 'individual', 'mortgage', 'rent', 'own', 'other', 'term60mths', 'vstatusverified', 'vstatusnotverified', 'lstatus'], dtype='object')

In [29]:
# transform the data sets

df2 = encoder.transform(df)

In [30]:
df2.head()

,annualinc,collections,delinq,inq,openacc,dti,pubrec,lstatus,applytype_INDIVIDUAL,homeowner_MORTGAGE,homeowner_RENT,homeowner_OWN,term_36 months,vstatus_Not Verified,vstatus_Verified
0,55.00,0,0,0,33,25.66,0,On Time,1,1,0,0,1,1,0
1,130.00,0,1,0,6,4.90,0,On Time,1,0,1,0,1,0,1
2,52.60,0,0,0,9,15.26,0,On Time,1,0,0,1,1,0,1
3,44.00,0,2,1,23,24.66,0,On Time,1,0,1,0,1,0,0
4,85.00,0,0,1,13,7.84,0,On Time,1,1,0,0,0,0,1


In [31]:
# we can retrieve the feature names as follows:

encoder.get_feature_names_out()

['annualinc',
 'collections',
 'delinq',
 'inq',
 'openacc',
 'dti',
 'pubrec',
 'lstatus',
 'applytype_INDIVIDUAL',
 'homeowner_MORTGAGE',
 'homeowner_RENT',
 'homeowner_OWN',
 'term_36 months',
 'vstatus_Not Verified',
 'vstatus_Verified']

In [33]:
#df2.to_csv("validdata1.csv", index=False)

**==============================================================================================================**

## Label Encoding

Label Encoding is a popular encoding technique for handling categorical variables. In this technique, each label is assigned a unique integer based on alphabetical ordering.

label_encoder object knows how to understand word labels. 

`label_encoder = LabelEncoder()`

Encode labels in column 'Country'. 

`df['Timely'] = label_encoder.fit_transform(df['Timely'])` 


In [ ]:
df.columns

In [ ]:
df.lstatus.value_counts()

In [ ]:
#df['species'] = df['species'].astype("category")

In [ ]:
#df.species.value_counts()

In [ ]:
# label_encoder object knows how to understand word labels. 
label_encoder = LabelEncoder()

# Encode labels in column 'Country'. 

df['lstatus'] = label_encoder.fit_transform(df['lstatus'])

In [ ]:
df["lstatus"].value_counts()

In [ ]:
df.head()

In [ ]:
#df.to_csv("calibdata1.csv", index=False)

**==============================================================================================================**

**==============================================================================================================**

**==============================================================================================================**

# Feature Scaling

## Data Standardization
<p>
Data is usually collected from different agencies in different formats.
(Data standardization is also a term for a particular type of data normalization where we subtract the mean and divide by the standard deviation.)
</p>

### What is standardization?

Standardisation involves centering the variable at zero, and standardising the variance to 1. The procedure involves subtracting the mean of each observation and then dividing by the standard deviation:

**z = (x - x_mean) /  std**

The result of the above transformation is **z**, which is called the z-score, and represents how many standard deviations a given observation deviates from the mean. A z-score specifies the location of the observation within a distribution (in numbers of standard deviations respect to the mean of the distribution). The sign of the z-score (+ or - ) indicates whether the observation is above (+) or below ( - ) the mean.

The shape of a standardised (or z-scored normalised) distribution will be identical to the original distribution of the variable. If the original distribution is normal, then the standardised distribution will be normal. But, if the original distribution is skewed, then the standardised distribution of the variable will also be skewed. In other words, **standardising a variable does not normalize the distribution of the data** and if this is the desired outcome, we should implement any of the techniques discussed in section 7 of the course.

In a nutshell, standardisation:

- centers the mean at 0
- scales the variance at 1
- preserves the shape of the original distribution
- the minimum and maximum values of the different variables may vary
- preserves outliers

Good for algorithms that require features centered at zero.

### Feature magnitude matters because:

- The regression coefficients of linear models are directly influenced by the scale of the variable.
- Variables with bigger magnitude / larger value range dominate over those with smaller magnitude / value range
- Gradient descent converges faster when features are on similar scales
- Feature scaling helps decrease the time to find support vectors for SVMs
- Euclidean distances are sensitive to feature magnitude.
- Some algorithms, like PCA require the features to be centered at 0.


### The machine learning models affected by the feature scale are:

- Linear and Logistic Regression
- Neural Networks
- Support Vector Machines
- KNN
- K-means clustering
- Linear Discriminant Analysis (LDA)
- Principal Component Analysis (PCA)


**Feature scaling** refers to the methods or techniques used to normalize the range of independent variables in our data, or in other words, the methods to set the feature value range within a similar scale. Feature scaling is generally the last step in the data preprocessing pipeline, performed **just before training the machine learning algorithms**.

There are several Feature Scaling techniques, which we will discuss throughout this section:

- Standardisation
- Mean normalisation
- Scaling to minimum and maximum values - MinMaxScaling
- Scaling to maximum value - MaxAbsScaling
- Scaling to quantiles and median - RobustScaling
- Normalization to vector unit length

| Name | Sklearn_class |
|-------------|------------|
|Standard scaler | Standard scaler | 
|MinMaxScaler    | MinMax Scaler   |
|MaxAbs Scaler   | MaxAbs Scaler   |
|Robust scaler   | Robust scaler   |
|Quantile Transformer_Normal | Quantile Transformer(output_distribution ='normal')|
|Quantile Transformer_Uniform| Quantile Transformer(output_distribution = 'uniform')|
|PowerTransformer-Yeo-Johnson| PowerTransformer(method = 'yeo-johnson')|
|Normalizer | Normalizer|

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
X = df.iloc[:, 0:5]
y = df.iloc[:, 5]

In [ ]:
X.values, y.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# standardisation: with the StandardScaler from sklearn

# set up the scaler
scaler = StandardScaler()

In [ ]:
# fit the scaler to the train set, it will learn the parameters
scaler.fit(X)

In [ ]:
X_scaled = scaler.fit_transform(X)

In [ ]:
X_scaled

In [ ]:
X_scaled.shape

In [ ]:
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

In [ ]:
X_scaled

In [ ]:
y

In [ ]:
y.shape

In [ ]:
# fit the scaler to the train set, it will learn the parameters
scaler.fit(X_train)

In [ ]:
# transform train and test sets
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_scaled

In [ ]:
X_test_scaled

In [ ]:
combined = np.concatenate((X_scaled, y), axis=0)
combined

In [ ]:
combined = np.concatenate((X_train_scaled, X_test_scaled), axis=0)
combined

In [ ]:
X_scaled = pd.DataFrame(combined, columns=X.columns)

In [ ]:
X_scaled

In [ ]:
X_scaled.describe()

In [ ]:
X_remain = df.iloc[:, 7:]
X_remain

In [ ]:
df2 = pd.concat([X_scaled, y],axis=1)

In [ ]:
df2

In [ ]:
#df2.to_csv("winemod.csv", index=False)

### Merging all data

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.drop(['lotarea', 'masvnrarea', 'bsmtfinsf1', 'bsmtfinsf2', 'bsmtunfsf', 'totalbsmtsf', 
         '1stflrsf', '2ndflrsf', 'grlivarea', 'bsmtfullbath', 'bsmthalfbath', 'fullbath', 
         'halfbath', 'bedroomabvgr', 'kitchenabvgr', 'totrmsabvgrd', 'fireplaces', 
         'garagecars', 'garagearea', 'years', 'saleprice'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
df3 = pd.concat([df,df2], axis=1)

In [ ]:
df3

In [ ]:
#df3.to_csv("ameshousingmod.csv", index=False)

## Scaling to Minimum and Maximum values - MinMaxScaling

Minimum and maximum scaling squeezes the values between 0 and 1. It subtracts the minimum value from all the observations, and then divides it by the value range:

X_scaled = (X - X.min / (X.max - X.min)


The result of the above transformation is a distribution which values vary within the range of 0 to 1. But the mean is not centered at zero and the standard deviation varies across variables. The shape of a min-max scaled distribution will be similar to the original variable. This scaling technique is also sensitive to outliers.

In a nutshell, MinMaxScaling:

- the minimum and maximum values are 0 and 1.
- does not center the mean at 0
- variance varies across variables
- sensitive outliers

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
X = df.iloc[:, 16:21]
y = df.iloc[:, 21:]

In [ ]:
X.values, y.values

In [ ]:
# set up the scaler
minmax = MinMaxScaler()

In [ ]:
# fit the scaler to the train set, it will learn the parameters
minmax.fit(X)

In [ ]:
# transform train and test sets
X_scaled = minmax.transform(X)

In [ ]:
X_scaled[0:5]

In [ ]:
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

In [ ]:
X_scaled

In [ ]:
X_scaled.describe()

In [ ]:
df2 = pd.concat([X_scaled,y], axis=1)

In [ ]:
df2

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.drop(['carsowned', 'children', 'totalchildren', 'income', 'age', 'buyer'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
df3 = pd.concat([df,df2], axis=1)

In [ ]:
df3

In [ ]:
#df3.to_csv("bicycleclassific.csv", index=False)

## Mean Normalisation


Mean normalisation involves centering the variable at zero, and re-scaling to the value range. The procedure involves subtracting the mean of each observation and then dividing by difference between the minimum and maximum value:

**x_scaled = (x - x_mean) / ( x_max - x_min)**


The result of the above transformation is a distribution that is centered at 0, and its minimum and maximum values are within the range of -1 to 1. The shape of a mean normalised distribution will be similar to the original distribution.

In a nutshell, mean normalisation:

- centers the mean at 0
- variance will be different
- the minimum and maximum values are squeezed between -1 and 1
- preserves outliers

Good for algorithms that require features centered at zero.


## Scaling to maximum value - MaxAbsScaling

Maximum absolute scaling scales the data to its absolute maximum value:

X_scaled = X / abs(X.max)

The result of the above transformation is a distribution which values vary within the range of -1 to 1. But the mean is not centered at zero and the standard deviation varies across variables.

Scikit-learn suggests that this transformer is meant for data that is centered at zero, and for sparse data.

## Scaling to quantiles and median - RobustScaling

In this procedure the median is removed from the observations and then they are scaled to the inter-quantile range (IQR). The IQR is the range between the 1st quartile (25th quantile) and the 3rd quartile (75th quantile).

X_scaled = X - X_median / ( X.quantile(0.75) - X.quantile(0.25) )

This robust scaling method produces more robust estimates for the center and range of the variable, and is recommended if the data shows outliers.

## Scaling to vector unit  length / unit norm

In this procedure we scale the components of a feature vector such that the complete vector has a length of 1 or, in other words a norm of 1. **Note** that this normalisation procedure normalises the **feature** vector, and not the **observation** vector. So we divide by the norm of the feature vector, observation per observation, across the different variables, and not by the norm of the **observation** vector, across observations for the same feature.

First, let me give you the formulas, and then I illustrate with an example.

### Scaling to unit norm, formulas

Scaling to unit norm is achieved by dividing each feature vector by either the Manhattan distance (l1 norm) or the Euclidean distance of the vector (l2 norm):

X_scaled_l1 = X / l1(X)

X_scaled_l2 = X / l2(X)


The **Manhattan distance** is given by the sum of the absolute components of the vector:

l1(X) = |x1| + |x2| + ... + |xn|


Whereas the **Euclidean distance** is given by the square root of the square sum of the component of the vector:

l2(X) = sqr( x1^2 + x2^2 + ... + xn^2 )


In the above example, x1 is variable 1, x2 variable 2, and xn variable n, and X is the data for 1 observation across variables (a row in other words).

**Note** as well that as the euclidean distance squares the values of the feature vector components, outliers have a heavier weight. With outliers, we may prefer to use l1 normalisation.


### Scaling to unit norm, examples

For example, if our data has 1 observations (1 row) and 3 variables:

- number of pets
- number of children
- age

The values for each variable for that single observation are 10, 15 and 20. Our vector X = [10, 15, 20]. Then:

l1(X) = 10 + 15 + 20 = 45

l2(X) = sqr( 10^2 + 15^2 + 20^2) = sqr( 100 + 225 + 400) = **26.9**

The euclidean distance is always smaller than the manhattan distance.


The normalised vector values are therefore:

X_scaled_l1 = [ 10/45, 15/45, 20/45 ]      =  [0.22, 0.33, 0.44]

X_scaled_l2 = [10/26.9, 15/26.9, 20/26.9 ] =  [0.37, 0.55, 0.74]


Scikit-learn recommends this scaling procedures for text classification or clustering. For example, they quote the dot product of two l2-normalized TF-IDF vectors is the cosine similarity of the vectors and is the base similarity metric for the Vector Space Model commonly used by the Information Retrieval community.

**==============================================================================================================**

**==============================================================================================================**

#### Python code done by Dennis Lam